In [1]:
import sys
import subprocess
sys.path.append('/efs/SHIFT-Python-Utilities/')
from shift_python_utilities.intake_shift import shift_catalog

import numpy as np
import rioxarray as rxr
import xarray as xr
import rasterio as rio

import odc.geo.xr
if odc.geo.__version__ != '0.4.1':
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'odc-geo==0.4.1'])

In [2]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:33635")
client

<Client: 'tcp://127.0.0.1:33635' processes=6 threads=6, memory=23.02 GiB>

In [3]:
cat = shift_catalog()
ds = cat.aviris_v1_gridded.read_chunked()
ds.rio.write_crs(rio.CRS.from_wkt(",".join(ds.attrs['coordinate system string'])), inplace=True)
ds

<xarray.Dataset>
Dimensions:      (time: 13, y: 12023, wavelength: 425, x: 13739)
Coordinates:
    spatial_ref  int64 0
  * time         (time) datetime64[us] 2022-02-24 2022-02-28 ... 2022-05-29
  * wavelength   (wavelength) float32 377.2 382.2 387.2 ... 2.496e+03 2.501e+03
  * x            (x) float64 7.177e+05 7.177e+05 ... 7.864e+05 7.864e+05
  * y            (y) float64 3.866e+06 3.866e+06 ... 3.806e+06 3.806e+06
Data variables:
    reflectance  (time, y, wavelength, x) float32 dask.array<chunksize=(1, 1, 425, 13739), meta=np.ndarray>
Attributes: (12/13)
    description:               flight_products/20220224/box_mosaics/box_rfl_p...
    samples:                   13739
    lines:                     12023
    bands:                     425
    header offset:             0
    file type:                 ENVI Standard
    ...                        ...
    interleave:                bil
    byte order:                0
    map info:                  ['UTM', '1', '1', '717720', '3865865', '5', '5...
    coordinate system string:  ['PROJCS["WGS_1984_UTM_Zone_10N"', 'GEOGCS["GC...
    wavelength:                ['377.1956495', '382.20564950000005', '387.215...
    fwhm:                      ['5.57', '5.58', '5.58', '5.58', '5.5900000000...

In [4]:
sub = ds.isel(time=0).reflectance.transpose('wavelength', 'y', 'x')
sub.data

/tmp/ipykernel_4411/1248270608.py:1: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by converting the values to nanosecond precision ahead of time.
  sub = ds.isel(time=0).reflectance.transpose('wavelength', 'y', 'x')


dask.array<transpose, shape=(425, 12023, 13739), dtype=float32, chunksize=(425, 1, 13739), chunktype=numpy.ndarray>

In [5]:
how = sub.odc.output_geobox(32610, resolution=(500))
sub = sub.odc.reproject(how=how, resampling='nearest')
sub

<xarray.DataArray 'reproject-30c8e7f043fd4191b87ee33488b2eeba' (
                                                                wavelength: 425,
                                                                y: 121, x: 138)>
dask.array<reproject, shape=(425, 121, 138), dtype=float32, chunksize=(425, 1, 138), chunktype=numpy.ndarray>
Coordinates:
    spatial_ref  int32 32610
    time         datetime64[ns] 2022-02-24
  * wavelength   (wavelength) float32 377.2 382.2 387.2 ... 2.496e+03 2.501e+03
  * y            (y) float64 3.866e+06 3.865e+06 ... 3.806e+06 3.806e+06
  * x            (x) float64 7.178e+05 7.182e+05 ... 7.858e+05 7.862e+05

In [6]:
%%time
sub.compute()

CPU times: user 4.16 s, sys: 548 ms, total: 4.71 s
Wall time: 37min 38s


<xarray.DataArray 'reproject-30c8e7f043fd4191b87ee33488b2eeba' (
                                                                wavelength: 425,
                                                                y: 121, x: 138)>
array([[[    0.,     0.,     0., ...,     0.,     0.,     0.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        ...,
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [    0.,     0.,     0., ...,     0.,     0.,     0.],
        [    0.,     0.,     0., ...,     0.,     0.,     0.]],

       [[    0.,     0.,     0., ...,     0.,     0.,     0.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        ...,
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [    0.,     0.,     0., ...,     0.,     0.,     0.],
        [    0.,     0.,     0., ...,     0.,     0.,     0.]],

       [[    0.,     0.,     0., ...,     0.,     0.,     0.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        ...,
...
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [    0.,     0.,     0., ...,     0.,     0.,     0.],
        [    0.,     0.,     0., ...,     0.,     0.,     0.]],

       [[    0.,     0.,     0., ...,     0.,     0.,     0.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        ...,
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [    0.,     0.,     0., ...,     0.,     0.,     0.],
        [    0.,     0.,     0., ...,     0.,     0.,     0.]],

       [[    0.,     0.,     0., ...,     0.,     0.,     0.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        ...,
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [    0.,     0.,     0., ...,     0.,     0.,     0.],
        [    0.,     0.,     0., ...,     0.,     0.,     0.]]],
      dtype=float32)
Coordinates:
    spatial_ref  int32 32610
    time         datetime64[ns] 2022-02-24
  * wavelength   (wavelength) float32 377.2 382.2 387.2 ... 2.496e+03 2.501e+03
  * y            (y) float64 3.866e+06 3.865e+06 ... 3.806e+06 3.806e+06
  * x            (x) float64 7.178e+05 7.182e+05 ... 7.858e+05 7.862e+05